In [1]:
from langchain.callbacks import StdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os
from langchain_community.llms.cloudflare_workersai import CloudflareWorkersAI
from langchain_community.llms.tongyi import Tongyi
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv(override=True)

account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')
print(account_id)
print(api_token)

# CloudflareWorkersAI
model = '@cf/meta/llama-3-8b-instruct'
cf_llm = CloudflareWorkersAI(
    account_id=account_id,
    api_token=api_token,
    model=model
)

DASHSCOPE_API_KEY = os.getenv('DASHSCOPE_API_KEY')
print(DASHSCOPE_API_KEY)

# qwen
qw_llm = Tongyi(
    model='qwen2-1.5b-instruct'
)

# qwen 兼容 openai的接口
qw_llm_openai = ChatOpenAI(
    openai_api_base='https://dashscope.aliyuncs.com/compatible-mode/v1',
    openai_api_key=DASHSCOPE_API_KEY,
    model_name="qwen2-1.5b-instruct",
    temperature=0.7,
    streaming=True,
)

api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')
print(api_key)
print(base_url)

# openai/moonshot
ms_llm = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

8483c3ec7a0cbc54a8d660b5b9002b04
Gcllof8ze6dgtcqFI5FQZ2SD_5tfCD4Db7NuS6jn
sk-01c5003340c3453b934052d737d45e01
sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


In [7]:
# 内置回调处理器 StdOutCallbackHandler

handler = StdOutCallbackHandler()
prompt = PromptTemplate.from_template("Who is {name}?")
chain = LLMChain(llm=qw_llm_openai, prompt=prompt, callbacks=[handler])
# chain = prompt | qw_llm_openai | lambda x: callbacks=[handler]
chain.invoke("Super Mario")

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'langchain_core.callbacks.stdout.StdOutCallbackHandler'>

In [8]:
# 自定义回调处理器
# 我们来实现一个处理器，统计每次 LLM 交互的处理时间。
from langchain.callbacks.base import BaseCallbackHandler
import time


class TimerHandler(BaseCallbackHandler):

    def __init__(self) -> None:
        super().__init__()
        self.previous_ms = None
        self.durations = []

    def current_ms(self):
        return int(time.time() * 1000 + time.perf_counter() % 1 * 1000)

    def on_chain_start(self, serialized, inputs, **kwargs) -> None:
        self.previous_ms = self.current_ms()

    def on_chain_end(self, outputs, **kwargs) -> None:
        if self.previous_ms:
            duration = self.current_ms() - self.previous_ms
            self.durations.append(duration)

    def on_llm_start(self, serialized, prompts, **kwargs) -> None:
        self.previous_ms = self.current_ms()

    def on_llm_end(self, response, **kwargs) -> None:
        if self.previous_ms:
            duration = self.current_ms() - self.previous_ms
            self.durations.append(duration)

In [10]:
timerHandler = TimerHandler()
prompt = PromptTemplate.from_template("What is the HEX code of color {color_name}?")
chain = LLMChain(llm=qw_llm_openai, prompt=prompt, callbacks=[timerHandler])
response = chain.invoke("blue")
print(response)
response = chain.invoke("purple")
print(response)

timerHandler.durations

{'color_name': 'blue', 'text': 'The hexadecimal (HEX) code for the color blue is #008000.'}
{'color_name': 'purple', 'text': 'The hexadecimal (HEX) code for purple is #800080.'}


[1381, -11]

In [12]:
# Model 与 callbacks
# callbacks 可以在构造函数中指定，也可以在执行期间的函数调用中指定。
timerHandler = TimerHandler()
qw_llm_openai_2 = ChatOpenAI(
    openai_api_base='https://dashscope.aliyuncs.com/compatible-mode/v1',
    openai_api_key=DASHSCOPE_API_KEY,
    model_name="qwen2-1.5b-instruct",
    temperature=0.7,
    streaming=True,
    callbacks=[timerHandler]
)
response = qw_llm_openai_2.invoke("What is the HEX code of color BLACK?")
print(response)

timerHandler.durations

content='The hexadecimal (HEX) code for black is #000000.' response_metadata={'finish_reason': 'stop'} id='run-0f76c9cf-9d6e-4d32-847f-18353df050f9-0'


[503]

In [15]:
## 也可以这样传参
response = qw_llm_openai_2.invoke("What is the HEX code of color BLACK?",
                                  config={"callbacks": [timerHandler]})
print(response)

timerHandler.durations

content='The hexadecimal (HEX) code for black is #000000.' response_metadata={'finish_reason': 'stop'} id='run-784923c2-79ec-4311-9e10-e8c18b7aa72c-0'


[503, 5511]